# Catatan:
Monggo bisa nyoba nambah model di pipeline make classifier lainnya dengan parameter atau feature yang beda-beda. Parameter SVM-nya berdasarkan paper yang disitasi.

File testing-nya gak ada labelnya jadi buat evaluasi akurasi pake hold out atau k-fold di training

Nuwuuuuuun.


---



# Author Profiling
A final project for KS SC: NLP class by:
1.   Alvin Romadhon
2.   Andri Immanudin
3.   Desthalia
4.   Gama Candra T. K.


This work is an attemp to recreate work based on:
> Basile, A., Dwyer, G., Medvedeva, M., Rawee, J., Haagsma, H., & Nissim, M. (2017). N-GrAM: New Groningen Author-profiling Model. *CoRR, abs/1707.03764.*

*All files can be accessed on [this Google Drive](https://drive.google.com/drive/folders/1cbEc24TJWsw80WcXKDZ1HXB3OU8oTz9Z).*



---

## A. Importing libraries

In [0]:
!pip install -U -q PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth, files
from oauth2client.client import GoogleCredentials

import glob
import pandas as pd
import re

from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.svm import LinearSVC

## B. Downloading all files

In [0]:
#Folder's IDs for each test and training folder

nama = {'train_nl': '1g9pau51NnPanw9NUcVd1_OP7BfdYiCBh',
        'train_en': '1A6nHq1picLrXMyNF4xMw-vFxTP2STQ6r',
        'train_es': '162c0Szb5IffXR7jtSQUJczoV_UoXVxDy',
        'test_nl': '15h8cy1AyJ6w-pxy7b3XD8e5aOXrLErpw',
        'test_en': '1gHdNyh29E4qKY66Es4vkXg4Q6JYPuuPO',
        'test_es': '1TIo79hmG5TcrYe-U1g9tXQl10aVvj4FE'}

In [0]:
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Choose a local (colab) directory to store the data
current_dir = "~/"
for key, val in nama.items():
  local_download_path = os.path.expanduser(os.path.join(current_dir, key))
  try: 
    os.makedirs(local_download_path)
  except: pass
                                           
  # 2. Auto-iterate using the query syntax
  #    https://developers.google.com/drive/v2/web/search-parameters
  file_list = drive.ListFile({'q': "'{}' in parents".format(val)}).GetList()

  # 3. Create & download by ID
  print("Downloading to {}".format(local_download_path))
  for f in file_list:
    f_ = drive.CreateFile({'id': f['id']})
    fname = os.path.join(local_download_path, f['title'])
    f_.GetContentFile(fname, mimetype='text/csv')
  print("Done!")

Done!
Done!
Done!
Done!
Done!
Done!


## C. Preprocessing

Includes:
1.   Combining all csvs
2.   Removal of special characters, numbers, and link


In [0]:
#Combine all CSVs from each folder

#Train
train = {}
train_ = ['train_nl', 'train_en', 'train_es']

for t in train_:  
  path = os.path.join("/root/", t)
  files = glob.glob(path + "/*.csv")
  
  df = pd.DataFrame()
  for f in files:
    try:
      tmp = pd.read_csv(f, usecols=['tgl', 'gen'])
      df = df.append(tmp)
    except: 
      continue
  train[t] = df.reset_index(drop = True)
  

In [0]:
#Remove mentions, punctuations, link, hashtag, and numbers
for t in train:
  train[t]['tgl_bersih'] = train[t]['tgl'].str.replace(
      r'(b\")|(b\')|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|(\w*\d\w*)', 
      '')

## D. Feature Extraction and Modeling
Some preprocessing steps were also done in this section, mainly tokenizing and case folding. For feature extraction, we used two feature vectors: character n-gram (n = 3-5) and word n-gram. The classifier we used is Linear SVM.

In [0]:
svm = Pipeline([('features', FeatureUnion([
    ('char',TfidfVectorizer(ngram_range=(3,6), analyzer = 'char', min_df=2,
                            lowercase=True, sublinear_tf=True, use_idf=True,
                           norm='l2', binary = False)),
    ('word', TfidfVectorizer(analyzer = 'word', lowercase=True, min_df=2, 
                             sublinear_tf=True, norm='l2', binary = False))
    ])),
    ('clf', LinearSVC(C=1.0))])

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

rfc = Pipeline([('features', FeatureUnion([
    ('char',TfidfVectorizer(ngram_range=(3,6), analyzer = 'char', min_df=2,
                            lowercase=True, sublinear_tf=True, use_idf=True,
                           norm='l2', binary = False)),
    ('word', TfidfVectorizer(analyzer = 'word', lowercase=True, min_df=2, 
                             sublinear_tf=True, norm='l2', binary = False))
    ])),
    ('clf', RandomForestClassifier())])

abc = Pipeline([('features', FeatureUnion([
    ('char',TfidfVectorizer(ngram_range=(3,6), analyzer = 'char', min_df=2,
                            lowercase=True, sublinear_tf=True, use_idf=True,
                           norm='l2', binary = False)),
    ('word', TfidfVectorizer(analyzer = 'word', lowercase=True, min_df=2, 
                             sublinear_tf=True, norm='l2', binary = False))
    ])),
    ('clf', AdaBoostClassifier())])

In [0]:
from sklearn.naive_bayes import GaussianNB

gnb = Pipeline([('features', FeatureUnion([
    ('char',TfidfVectorizer(ngram_range=(3,6), analyzer = 'char', min_df=2,
                            lowercase=True, sublinear_tf=True, use_idf=True,
                           norm='l2', binary = False)),
    ('word', TfidfVectorizer(analyzer = 'word', lowercase=True, min_df=2, 
                             sublinear_tf=True, norm='l2', binary = False))
    ])),
    ('clf', GaussianNB())])


## E. Training and Evaluation

In [0]:
%%time 
# SVM
for t in train:
  X_train = train[t]['tgl_bersih']
  y_train = train[t]['gen']
#   pipeline.fit(X_train, y_train)
  score = cross_val_score(svm, X_train, y_train, cv=5).mean()
  
  print("Score {}: {}".format(t, score))

Score train_nl: 0.5495080251763877
Score train_en: 0.6053316553197199
Score train_es: 0.5566900442539706
CPU times: user 8min 46s, sys: 3.94 s, total: 8min 50s
Wall time: 8min 50s


In [0]:
%%time
# RFC
for t in train:
  X_train = train[t]['tgl_bersih']
  y_train = train[t]['gen']
#   pipeline.fit(X_train, y_train)
  score = cross_val_score(rfc, X_train, y_train, cv=5).mean()
  
  print("Score {}: {}".format(t, score))

Score train_nl: 0.5440745670647903
Score train_en: 0.5818055448843391
Score train_es: 0.54053709816234
CPU times: user 43min 35s, sys: 3.02 s, total: 43min 38s
Wall time: 43min 40s


In [0]:
%%time
# adaboost
for t in train:
  X_train = train[t]['tgl_bersih']
  y_train = train[t]['gen']
#   pipeline.fit(X_train, y_train)
  score = cross_val_score(abc, X_train, y_train, cv=5).mean()
  
  print("Score {}: {}".format(t, score))

Score train_nl: 0.5452319837656497
Score train_en: 0.5728941259633
Score train_es: 0.5706681313266369
CPU times: user 47min 34s, sys: 4.62 s, total: 47min 38s
Wall time: 47min 39s


Kesimpulannya, bahwa SVM dari segi running time beserta akurasi jauh lebih akurat dan lebih sedikit running timenya

## F. Testing

In [0]:
#Test

predict = {}
test_ = ['test_nl', 'test_en', 'test_es']

for e in test_:
  path = os.path.join("/root/", e)
  files = glob.glob(path + "/*.csv")
  
  for f in files:
    try:
      df = pd.read_csv(f)
      X_test = df[df.columns[1]]
      
      predict[x] = svm.predict(X_test)
    except: 
      continue

In [0]:
#
path = os.path.join("/root/", "test_en")
files_en = glob.glob(path + "/*.csv")
predict_en = []

for f in files_en:
  try:
    df = pd.read_csv(f)
    X_test = df[df.columns[1]]

    predict_en.append(svm.predict(X_test))
  except:
    continue

In [0]:
predict_en

[]